앞 장에서 사용한 수치 미분을 이용한 매개변수의 갱신은 단순하고 구현하기도 쉽지만 계산 시간이 오래 걸린다는 것이 단점이다. 이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 "__오차역전파법(backpropagation)__"을 배워보자.

오차역전파법을 제대로 이해하는 방법은 두가지가 있을 것이다. 하나는 수식을 통한 것이고, 다른 하나는 계산 그래플르 통한 것이다. 전자 쪽이 일반적인 방법으로, 특히 기계학습을 다루는 책 대부분은 수식을 중심으로 이야기를 전개한다. 확실히 수식을 사용한 설명은 정확하고 간결하므로 올바른 방법이라 할 수 있다. 하지만 이 방법으로는 본질을 놓치거나, 수많은 수식에 당황하는 일이 벌어지기도 한다. 그래서 이번 장에서는 계산 그래프를 사용해서 "시각적"으로 이해해보자. 

참고로 위 내용은 안드레 카패시(Andrej Karpathy)의 블로그, 페이페이 리(Fei-Fei Li) 교수가 진행한 딥러닝 수업 CS231n을 참고했다.

우선 계산 그래프란, 계산 과정을 그래프로 나타낸 것이다. 여기에서의 그래프는 우리가 잘 아는 그래프 자료구조로, 복수의 "__노드(Node)__"와 "__에지(edge)__"로 표현된다.(노드 사이의 직선을 에지라고 한다.)

# 5.1.1 계산 그래프로 풀다

그럼 간단한 문제를 계산 그래프로 풀어보자. 곧 보게 될 문제는 암산으로도 풀 정도로 간단하지만 지금의 목적은 계산 그래프에 익숙해지는 것이다. 

##### 문제 1: 현빈이는 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세 10% 부과된다.

계산 그래프의 각 과정은 노드와 화살표로 표현한다. 노드는 원(o)으로 표기하고 원 안에 연산 내욜을 적는다. 또, 계산 결과를 화살표 위에 적어 각 노드의 계산 결과가 왼쪽에서 오른쪽으로 전해지게 한다. 문제 1을 계산 그래프로 풀면 다음과 같이 된다.

<img src=images/5_01.png height=100px width=500px>

처음에는 사과의 100원이 "$\times 2$"노드를 흐르고, 200원이 되어 다음 노드로 전달된다. 이제 200원이 "$\times 1.1$"노드를 거쳐 220원이 된다. 따라서 이 계산 그래프에 따르면 최종 답은 220원이 된다.

또한, 위에서는 "$\times 2$"와 "$\times 1.1$"을 각각 하나의 연산으로 취급해 원 안에 표기했지만, 곱셈인 "$\times$"만을 연산으로 생각할 수도 있다. 이렇게 하면 밑의 그림과 같이 된다.

<img src=images/5_02.png height=100px width=500px>

이렇게 하면, "2"와 "1.1"은 각각 "사과의 개수"와 "소비세" 변수가 되어 원 밖에 표기하게 된다.

##### 문제 2: 현분이는 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개에 150원 있니다. 소비세가 10%일 때의 지불 금액을 구하세요.

이도 계산 그래프로 풀어보면 다음과 같다.

<img src=images/5_03.png height=100px width=500px>

이 문제에서는 덧셈 노드인 "+"가 새로 등장하여 사과와 귤의 금액을 합산한다. 계산 그래프는 왼쪽에서 오른쪽으로 계산을 진행한다. 회로에 전류가 흐르듯 계산 결과가 왼쪽에서 오른쪽으로 전달된다고 생각하면 된다.

계산 그래프를 이용한 문제풀이는 다음 흐름으로 진행된다.
1. 계산 그래프를 구성한다,
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.
여기서 "계산을 왼쪽에서 오른쪽으로 진행"하는 단계를 "__순전파(forward propagation)__"이라고 한다. 이는 계산 그래프의 출발점부터 종착점으로의 전파이다. 그리고 이를 반대로 진행하는 것이 "__역전파(backward propagation)__"이라고 한다. 이는 미분을 계산할 때 중요한 역할을 한다.

# 5.1.2 국소적 계산

계산 그래프의 특징은 "국소적 계산"을 전파함으로써 최종결괄르 얻는다는 점에 있다. 국소적 계산은 결국 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있다는 것이다. 예를 들어, 슈퍼마켓에서 사과 2개를 포함한 여러 식품을 구입하는 경우를 생각해보자. 이는 다음과 같은 계산 그래프로 나타낼 수 있다.

<img src=images/5_04.png height=100px width=500px>

여러 식품을 구입하여 총금액이 4,000원이 되었다. 여기에서 핵심은 각 노드에서의 계산은 국소적 계산이라는 점이다. 가령 사과와 그 이와의 물품 값을 더하는 계산 (4,000 + 200 => 4,200)은 4,000이라는 숫자가 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 뜻이다. 각 노드는 자신과 관련한 계산 외에는 아무것도 신경 쓸게 없다.<br>
이처럼 계산 그래프는 국소적 계산에 집중한다. 전체 계산이 제아무리 복잡하더라도 각 단계에서 하는 일은 해당 노드의 "국소적 게산"이다. 국소적인 계산은 단순하지만, 그 결과를 전달 함으로써 전체를 구성하는 복잡한 계산을 해낼 수 있다.

# 5.1.3 왜 계산 그래프로 푸는가?

계산 그래프의 이점은 "국소적 계산"이다. 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화할 수 있다. 또다른 이점으로, 계산 그래프는 중간 계산 결과를 모두 보관할 수 있다. 실제 계산 그래프를 사용하는 가장 큰 이유는 역전파를 이용해서 "미분"을 효율적으로 게산할 수 있다는 점에 있다.

계산 그래프의 역전파를 설명하기 위해 문제 1을 다시 꺼내보자. 문제 1은 사과를 2개 사서 소비세를 포함한 최종 금액을 구하는 것이다. 여기서 가령 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지를 알고 싶다고 하자. 이는 "사과 가격에 대한 지불 금액의 미분"을 구하는 문제에 해당한다. 기호로 나타낸다면 사과 값을 x, 지불 금액을 L이라 했을 때 ${\partial L \over \partial x}$를 구하는 것이다. 이 미분 값은 사과 값이 "아주 조금" 올랐을 때 지불 금액이 얼마나 증가하느냐를 표시한 것이다.
"사과 가격에 대한 지불 금액의 미분"같은 값은 게산 그래프에서 역전파를 하면 구할 수 있다. 먼저 결과만을 나타내면 다음과 같이 계산 그래프 상의 역전파에 의해서 미분을 구할 수 있다.

<img src=images/5_05.png height=100px width=500px>

이와 같이 역전파는 순전파와는 반대 방향의 반대 방향의 화살표로 그린다. 이 전파는 "국소적 미분"을 전달하고 그 미분 값은 화살표의 아래에 적는다. 이 예에서 역전파는 오른쪽에서 왼쪽으로 "1=>1.1=>2.2"순으로 미분 값을 전달한다.사과가 1원 오르면 최종 금액은 2.2원 오른다는 뜻이다.<br>
여기서는 사과 가격에 대한 미분만 구했지만, "소비세에 대한 지불 금액의 미분"이나 "사과 개수에 대한 지불 금액의 미분"도 같은 순서로 구할 수 있다. 그리고 그때는 중간까지 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산할 수 있다. 이처럼 계산 그래프의 이점은 수전파와 역전파를 활용해서 각 변수의 미분을 효율적으로 구할 수 있다는 것이다.